# Processed data storage

## Imports

In [ ]:
import os

import pandas as pd

from src.utils.const import DATA_DIR

### Useful path to data

In [ ]:
ROOT_DIR = os.path.join(os.getcwd(), '..')
INTERIM_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'interim')
PROCESSED_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'processed')

## Import all interim .csv

In [ ]:
movies = pd.read_parquet(
    os.path.join(INTERIM_DIR, 'movies.parquet')
)

tags = pd.read_parquet(
    os.path.join(INTERIM_DIR, 'tags.parquet')
)

ratings = pd.read_parquet(
    os.path.join(INTERIM_DIR, 'ratings.parquet')
)

tmdb = pd.read_parquet(
    os.path.join(INTERIM_DIR, 'tmdb.parquet')
)

genome = pd.read_parquet(
    os.path.join(INTERIM_DIR, 'genome.parquet')
)

## final.csv

In [ ]:
final = (movies
         .merge(ratings, on='movieId', how='inner')
         .merge(tags, on='movieId', how='left')
         .merge(tmdb, on='movieId', how='inner')
         .merge(genome, on='movieId', how='inner'))

# Cleaning
final = (final
         .fillna({'tag_count': 0})
         .drop(columns='movieId'))

final.info()